In [5]:
import vl_convert as vlc
import json
import copy
import pandas as pd

In [6]:
vl_spec='''{
  "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
  "width": 500,
  "height": 300,
  "data": {
    "url": "https://raw.githubusercontent.com/vega/vega-datasets/main/data/us-10m.json",
    "format": {
      "type": "topojson",
      "feature": "counties"
    }
  },
  "transform": [{
    "lookup": "id",
    "from": {
      "data": {
        "url": "https://raw.githubusercontent.com/vega/vega-datasets/main/data/unemployment.tsv"
      },
      "key": "id",
      "fields": ["rate"]
    }
  }],
  "projection": {
    "type": "albersUsa"
  },
  "mark": "geoshape",
  "encoding": {
    "color": {
      "field": "rate",
      "type": "quantitative"
    }
  }
}
'''

In [7]:
normalize={'date':'date','Date':'date','day':'date','Day':'date','month':'month','Month':'month','year':'year','Year':'year'}
time_format={'date':'D','Date':'D','day':'D','Day':'D','month':'M','Month':'M','year':'Y','Year':'Y'}
def get_opposite_color(hex_color):
	hex_color = hex_color.lstrip('#')
	r = int(hex_color[0:2], 16)
	g = int(hex_color[2:4], 16)
	b = int(hex_color[4:6], 16)
	r = 255 - r
	g = 255 - g
	b = 255 - b
	r_hex = format(r, '02x')
	g_hex = format(g, '02x')
	b_hex = format(b, '02x')
	return f"#{r_hex}{g_hex}{b_hex}"
def remove_lonely_numbers(lst):
	result = []
	lst=sorted(lst)
	for i in range(len(lst)):
		if (i == 0):
			if (lst[i+1] - lst[i] == 1):  
				result.append(lst[i])
		if (i == len(lst) - 1):
			if(lst[i] - lst[i-1] == 1):
				result.append(lst[i])
		if  (i!=0 and i!=len(lst)-1):
			if (lst[i] - lst[i-1] == 1) | (lst[i+1] - lst[i] == 1):
				result.append(lst[i])
	return result

def generate_conditions(numlist,mainField):
	filtered_years = remove_lonely_numbers(numlist)
	print(filtered_years)
	ranges = []
	start = None
	for year in filtered_years:
		if start is None:
			start = year
		elif year + 1 not in filtered_years:
			end = year
			if start != end:
				ranges.append((start, end))
			start = None
	if start is not None:
		end = filtered_years[-1]
		if start != end:
			ranges.append((start, end))
	conditions = [f"({normalize[mainField]}(datum['{mainField}']) >= {start-1} && {normalize[mainField]}(datum['{mainField}']) <= {end-1})" for start, end in ranges ]
	return conditions

def GeoMark(vega,mainField, subField, mainType, subType, value1):
	# newcolor = get_opposite_color(vega["encoding"]["color"]["value"])
	newcolor='red'
	vega["layer"].append( {
		"mark": {"type": "point", "filled": True},
		"transform": [{"filter": f"datum['{mainField}'] == {value1}"}], 
		"encoding": {
			"color": {"value": newcolor},
			"size": {"value": 100},
			"x": {"type": mainType, "field": mainField, "axis": {}, "bin": False},
			"y": {"type": subType, "field": subField}
		}})
	return vega

def ScatterText(vega,mainField, subField, mainType, subType, value1,value2, text,degree=0):
	# newcolor = get_opposite_color(vega["encoding"]["color"]["value"])
	newcolor='black'
	size=14
	if degree!=0:
		size=100
	if mainType == 'temporal':
		vega["data"]["format"]= {"type": "csv", "parse": {f'{mainField}': f"date:'%{time_format[mainField]}'"}}
		vega["layer"].append( {
		"mark": {"type": "text", "filled": True,"fontSize":size,'angle':degree,'dy':-10},
		"transform": [{"filter": f"{normalize[mainField]}(datum['{mainField}']) == {value1} && {normalize[subField]}(datum['{subField}']) == {value2}"}],
		"encoding": {
			"text" :{"value": text},
			"color": {"value": newcolor},
			"x": {"type": mainType, "field": mainField},
			"y": {"type": subType, "field": subField},
			"fontSize":{"value":size}
		}})
	else:
		vega["layer"].append( {
			"mark": {"type": "text", "filled": True,"fontsize":size,'angle':degree,'dy':-10},
			"transform": [{"filter": f"datum['{mainField}'] == {value1} && datum['{subField}'] == {value2}"}], # type: ignore
			"encoding": {
				"text" :{"value": text},
				"color": {"value": newcolor},
				"x": {"type": mainType, "field": mainField},
				"y": {"type": subType, "field": subField},
				"fontSize":{"value":size}
			}})
	return vega

def ScatterColor(vega,mainField, subField, mainType, subType, value, condition,value1=0):
	# newcolor = get_opposite_color(vega["encoding"]["color"]["value"])
	newcolor='red'
	new_layer = copy.deepcopy(vega["layer"][0])
	if condition == 'equal':
		new_layer["transform"] = [{"filter": f"datum['{mainField}'] == {value} && datum['{subField}'] == {value1}"}]
	elif condition == 'above':
		new_layer["transform"] = [{"filter": f"datum['{subField}'] >= {value}"}]
	elif condition == 'below':
		new_layer["transform"] = [{"filter": f"datum['{subField}'] <= {value}"}]
	elif condition == 'above_and_below':
		new_layer["transform"] = [{"filter": f"datum['{subField}'] >= {value} && datum['{subField}'] <= {value1}"}]
	elif condition == 'left_bound':
		new_layer["transform"] = [{"filter": f"datum['{mainField}'] >= {value}"}]
	elif condition == 'right_bound':
		new_layer["transform"] = [{"filter": f"datum['{mainField}'] <= {value}"}]
	elif condition == 'left_and_right':
		new_layer["transform"] = [{"filter": f"datum['{mainField}'] >= {value} && datum['{mainField}'] <= {value1}"}]
	elif condition == 'compare':
		new_layer["transform"] = [{"filter": f"datum['{mainField}'] == {value} || datum['{mainField}'] == {value1}"}]

	new_layer["encoding"]["color"]={"value": 'red'}
	vega["layer"].append(new_layer)
	return vega
def ScatterOpacity(vega,mainField, subField, mainType, subType, value,condition,value1=0):
	vega["layer"][0]["encoding"]["opacity"]={"value": 0.1}
	new_layer = copy.deepcopy(vega["layer"][0])
	if condition == 'equal':
		new_layer["transform"] = [{"filter": f"datum['{mainField}'] == {value} && datum['{subField}'] == {value1}"}]
	elif condition == 'above':
		new_layer["transform"] = [{"filter": f"datum['{subField}'] >= {value}"}]
	elif condition == 'below':
		new_layer["transform"] = [{"filter": f"datum['{subField}'] <= {value}"}]
	elif condition == 'above_and_below':
		new_layer["transform"] = [{"filter": f"datum['{subField}'] >= {value} && datum['{subField}'] <= {value1}"}]
	elif condition == 'left_bound':
		new_layer["transform"] = [{"filter": f"datum['{mainField}'] >= {value}"}]
	elif condition == 'right_bound':
		new_layer["transform"] = [{"filter": f"datum['{mainField}'] <= {value}"}]
	elif condition == 'left_and_right':
		new_layer["transform"] = [{"filter": f"datum['{mainField}'] >= {value} && datum['{mainField}'] <= {value1}"}]
	elif condition == 'compare':
		new_layer["transform"] = [{"filter": f"datum['{mainField}'] == {value} || datum['{mainField}'] == {value1}"}]
				
	new_layer["encoding"]["opacity"]={"value": 1}
	vega["layer"].append(new_layer)
	return vega


In [8]:
png_data = vlc.vegalite_to_png(vl_spec=vl_spec, scale=2)
with open("Sample4.png", "wb") as f:
	f.write(png_data)

In [15]:
vl_json=json.loads(vl_spec)
vl_json["layer"] = [{ 'mark': vl_json['mark'], 'encoding': vl_json['encoding'] }]
vl_json=GeoMark(vl_json, "id", "rate", "quantitative", "quantitative", 2115)
# vl_json=ScatterText(vl_json, "Horsepower", "Miles_per_Gallon", "quantitative", "quantitative", 165, 15,'A')
# # vl_json=ScatterMark(vl_json, "Horsepower", "Miles_per_Gallon", "quantitative", "quantitative", 150, 18)
# # vl_json=ScatterText(vl_json, "Horsepower", "Miles_per_Gallon", "quantitative", "quantitative", 150, 18,'B')
del vl_json['mark'], vl_json['encoding']
print(vl_json)
png_data = vlc.vegalite_to_png(vl_spec=vl_json, scale=2)
with open("RetrieveValue-Geo-Mark.png", "wb") as f:
	f.write(png_data)

{'$schema': 'https://vega.github.io/schema/vega-lite/v5.json', 'width': 500, 'height': 300, 'data': {'url': 'https://raw.githubusercontent.com/vega/vega-datasets/main/data/us-10m.json', 'format': {'type': 'topojson', 'feature': 'counties'}}, 'transform': [{'lookup': 'id', 'from': {'data': {'url': 'https://raw.githubusercontent.com/vega/vega-datasets/main/data/unemployment.tsv'}, 'key': 'id', 'fields': ['rate']}}], 'projection': {'type': 'albersUsa'}, 'layer': [{'mark': 'geoshape', 'encoding': {'color': {'field': 'rate', 'type': 'quantitative'}}}, {'mark': {'type': 'point', 'filled': True}, 'transform': [{'filter': "datum['id'] == 2115"}], 'encoding': {'color': {'value': 'red'}, 'size': {'value': 100}, 'x': {'type': 'quantitative', 'field': 'id', 'axis': {}, 'bin': False}, 'y': {'type': 'quantitative', 'field': 'rate'}}}]}
